# Web Scraper (Creating the Corpus)

In [42]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from itertools import chain
import os
import json

In [43]:
def collapse(lst):
    return '' .join(chain.from_iterable(lst))

## Star News Fact Checker

In [49]:
pages = [0, 1, 2, 3, 4, 5]
all_links = []

for page in pages:
    print(page)
    url = 'https://www.the-star.co.ke/news/fact-checker/?limit=10&page=' + str(page) +'&partial=true'
    page = requests.get(url).json()
    soup = BeautifulSoup(page["html"], "html.parser")
    articles = soup.find_all('div',attrs={'class':'section-article section-article-has-comments'})
    for x in articles:
        links = soup.find_all('a', href=True, attrs={'class':'image image-loader'})
        for x in links:
            all_links.append(x['href'])
    time.sleep(10)

    
url = 'https://www.the-star.co.ke/news/fact-checker/'
page = requests.get(url)    
soup = BeautifulSoup(page.text, "html.parser")
articles =soup.find('div',attrs={'class':'article-load'}).find_all('div',attrs={'class':'section-article section-article-has-comments'})
for x in articles:
    links = soup.find_all('a', href=True, attrs={'image image-loader'})
    for x in links:
        all_links.append(x['href'])

0
1
2
3
4
5


In [50]:
len(all_links)

1591

In [51]:
clean_list = [link for link in all_links if not link.startswith('/cartoon/')]

no_duplicates = [] 
for link in clean_list: 
    if link not in no_duplicates: 
        no_duplicates.append(link) 

79

In [52]:
upper_frame = []

for link in no_duplicates:
    try:
        frame = []
        all_text = []
        link_url = 'https://www.the-star.co.ke' + link
        article_page = requests.get(link_url)
        soup = BeautifulSoup(article_page.text, "html.parser")
        title = soup.find('h1',attrs={'class':'article-title article-title-primary'}).text.strip()
        status = soup.find('h2',attrs={'class':'article-title article-title-secondary'}).text.strip()
        texts = soup.find_all('div',attrs={'class':'article-widget article-widget-text'})
        for x in texts:
            text_body = x.text.strip()
            all_text.append(text_body)

        text = collapse(all_text)
        print(title, status)
        frame.append([title,text,status])
        upper_frame.extend(frame)
        time.sleep(10)    
        
    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',link_url)
        print (error_type, 'Line:', error_info.tb_lineno)
    
    continue

Does wearing masks for a long period of time reduce your oxygen uptake? PARTLY TRUE
Four Std 8 pupils, two teachers have not succumbed to Covid-19 FALSE
PARTLY TRUE: Safaricom calling for registration of next of kin FACT CHECKER
Raila's doctors haven't advised him to take a break from work FALSE
Photo claiming journalist shot dead during Bobi Wine arrest not true FALSE
Image of Ruto with a panga as he addresses crowd is altered ALTERED
President Trump did not refer to Republicans as 'dumbest' voters FALSE
Poster advertising youth meeting 'hosted' by Ruto is fabricated FACT CHECK
Photo does not show Raila Odinga being chased away by a mob HOAX
Advert for free courses from the NYS is not legitimate FAKE
FALSE: Former Nairobi County Secretary Ayisi is not dead 
Advert asking ‘Sh200’ registration fee for Quickmart Supermarket jobs is fake HOAX
Two ladies spotted with Governor Oparanya during Mashujaa Day aren't his wives FACT CHECK
FACT CHECK: The Star's front page with Ruto supporting BBI

In [53]:
data=pd.DataFrame(upper_frame, columns=['Title','Text','Status'])
data.head()

,Title,Text,Status
0,Does wearing masks for a long period of time r...,Just like wearing a shoe when one leaves their...,PARTLY TRUE
1,"Four Std 8 pupils, two teachers have not succu...",Information circulating on social media that f...,FALSE
2,PARTLY TRUE: Safaricom calling for registratio...,A viral WhatsApp message claiming that Safaric...,FACT CHECKER
3,Raila's doctors haven't advised him to take a ...,A Facebook post claiming that ODM leader Raila...,FALSE
4,Photo claiming journalist shot dead during Bob...,A photo that has been circulating on social me...,FALSE


In [54]:
path = '/home/jaden/Development/Notebooks/kenyan-news'
data.to_csv(os.path.join(path,r'star-news.csv'))

## AFP Fact Check

In [74]:
afp_pages = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
afp_links = []

for page in afp_pages:
    print(page)
    url = 'https://factcheck.afp.com/afp-kenya?page=' + str(page)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    
    articles = soup.find('main', attrs={'role':'main'}).find_all('div',attrs={'class':'card'})
    for x in articles:
        links = soup.find_all('a', href=True)
        for y in links:
            if y.parent.name == 'div':
                afp_links.append(y['href'])
            
    time.sleep(10)



0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [75]:
clean_list = [link for link in afp_links if not link.startswith('#main')]
clean_list = [link for link in clean_list if not link == '/']

no_duplicates = [] 
for link in clean_list: 
    if link not in no_duplicates: 
        no_duplicates.append(link) 
        
len(no_duplicates)

84

In [77]:
no_duplicates

['/fraudulent-facebook-posts-claim-world-bank-giving-loans-small-businesses-africa',
 '/photos-show-honduran-migrants-trying-enter-guatemala-not-post-election-violence-uganda',
 '/these-pictures-do-not-show-torture-scenes-opposition-supporters-uganda',
 '/picture-shows-italian-performance-artist-vanessa-beecroft-and-orphaned-sudanese-twins-she-tried',
 '/video-shows-protest-south-africa-not-bobi-wine-supporters-uk',
 '/hoax-tweets-falsely-claim-support-biden-obama-ugandan-presidential-challenger',
 '/footage-bustling-intersection-comes-vietnam-not-kenya',
 '/image-injured-journalist-was-taken-during-protest-five-years-ago-not-recent-rally',
 '/not-real-icc-witness-statement-kenyan-lawyer-accused-bribery',
 '/these-are-old-images-ugandan-soldiers-kampala-after-us-warned-terror-threat',
 '/video-shows-protesters-storming-german-embassy-sudan-2012',
 '/videos-purporting-show-violence-during-recent-elections-zanzibar-are-old-and-unrelated',
 '/hoax-embassy-pages-falsely-claim-trump-made-vi

In [87]:
upper_frame = []

for link in no_duplicates:
    try:
        frame = []
        all_text = []
        link_url = 'https://factcheck.afp.com' + link
        article_page = requests.get(link_url)
        soup = BeautifulSoup(article_page.text, "html.parser")
        
        title = soup.find('div', attrs={'class':'content-header-single'}).find('h1',attrs={'class':'content-title'}).text.strip()
        texts = soup.find('div', attrs={'class':'article-entry clearfix'}).find_all('p')
        summary = soup.find('div', attrs={'class':'article-entry clearfix'}).find('h3').text.strip()
        all_text.append(summary)
        
        for x in texts:
            text_body = x.text.strip()
            all_text.append(text_body)

        text = collapse(all_text)
        text = re.sub(r'http\S+', '', text)
        status = 1 #misleading or false
        
        frame.append([title,text,status])
        print(title)
        upper_frame.extend(frame)
        time.sleep(10)    
        
    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',link_url)
        print (error_type, 'Line:', error_info.tb_lineno)
    
    continue

Fraudulent Facebook posts claim the World Bank is giving loans to small businesses in Africa
Photos show Honduran migrants trying to enter Guatemala, not post-election violence in Uganda
These pictures do not show torture scenes of opposition supporters in Uganda
This picture shows Italian performance artist Vanessa Beecroft and orphaned Sudanese twins she tried to adopt
This video shows a protest in South Africa, not Bobi Wine supporters in the UK
Hoax tweets falsely claim support from Biden, Obama for Ugandan presidential challenger
This footage of a bustling intersection comes from Vietnam, not Kenya
This image of an injured journalist was taken during a protest five years ago, not at a recent rally
This is not a real ICC witness statement from a Kenyan lawyer accused of bribery
These are old images of Ugandan soldiers in Kampala after the US warned of a terror threat
This video shows protesters storming the German embassy in Sudan in 2012
Videos purporting to show violence during r

In [88]:
data=pd.DataFrame(upper_frame, columns=['Title','Text','Status'])
data.head()

,Title,Text,Status
0,Fraudulent Facebook posts claim the World Bank...,Facebook posts inviting small business owners ...,1
1,Photos show Honduran migrants trying to enter ...,A post shared dozens of times on Facebook clai...,1
2,These pictures do not show torture scenes of o...,A Facebook post claiming to show images of tor...,1
3,This picture shows Italian performance artist ...,A bogus Facebook post claiming to show a white...,1
4,"This video shows a protest in South Africa, no...",A video viewed thousands of times has been sha...,1


In [89]:
path = '/home/jaden/Development/Notebooks/kenyan-news'
data.to_csv(os.path.join(path,r'afp-kenya.csv'))

## Pesa Check

In [1]:
! pip install selenium

     |████████████████████████████████| 904 kB 275 kB/s eta 0:00:01


In [83]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome()
pesa_links = []

browser.get("https://pesacheck.org/tagged/kenya")
time.sleep(1)

elem = browser.find_element_by_tag_name("body")
#750
no_of_pagedowns = 800

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    no_of_pagedowns -= 1
    
soup = BeautifulSoup(browser.page_source)

number = 0

posts = soup.find('div', attrs={'class':'js-tagStream'}).find_all('div', attrs={'class':'streamItem streamItem--postPreview js-streamItem'})

for post in posts:
    print(post['data-action-scope'])
    soup.find()
    link = post.find('div', attrs={'class':'postArticle-readMore'}).find('a', attrs={'class':'button button--smaller button--chromeless u-baseColor--buttonNormal'} , href=True)
    print(link['href'])
    pesa_links.append(link['href'])
    

_actionscope_2
https://pesacheck.org/false-this-tweet-of-kenyan-businessman-buzeki-criticising-the-jubilee-party-is-fake-5f5a30365a0c?source=---------0-----------------------
_actionscope_4
https://pesacheck.org/hoax-joyful-women-organization-is-not-taking-loan-applications-through-an-online-form-8013ba6ac91e?source=---------1-----------------------
_actionscope_6
https://pesacheck.org/altered-this-photo-of-kenyas-former-prime-minister-raila-odinga-holding-a-bible-upside-down-3d85a3093e31?source=---------2-----------------------
_actionscope_8
https://pesacheck.org/false-this-photo-does-not-show-men-in-nigerian-state-uniform-226292e49e47?source=---------3-----------------------
_actionscope_10
https://pesacheck.org/hoax-this-advertisement-for-jobs-at-caf%C3%A9-javas-uganda-is-fake-3e4cc3b6fdef?source=---------4-----------------------
_actionscope_12
https://pesacheck.org/hoax-this-poster-advertising-jobs-with-the-kenya-defence-forces-is-not-legitimate-95f3c806e93d?source=---------5----

https://pesacheck.org/true-a-kenya-airways-staffer-has-been-suspended-after-recording-a-chinese-plane-landing-in-kenya-64aceafe6f3d?source=---------596-----------------------
_actionscope_1196
https://pesacheck.org/hoax-this-post-advertising-funding-opportunities-at-usaid-is-fake-9398deae2270?source=---------597-----------------------
_actionscope_1198
https://pesacheck.org/hoax-this-ad-for-jobs-at-bata-kenya-is-fake-4ba8019e8e0d?source=---------598-----------------------
_actionscope_1200
https://pesacheck.org/hoax-this-list-ranking-how-kenyas-counties-have-used-the-national-government-affirmative-action-9288659a9eb3?source=---------599-----------------------
_actionscope_1202
https://pesacheck.org/true-nigeria-has-confirmed-the-first-case-of-covid-19-in-sub-saharan-africa-e8fa69e0d75c?source=---------600-----------------------
_actionscope_1204
https://pesacheck.org/hoax-this-post-claiming-to-show-chaos-at-a-bbi-meeting-in-meru-is-false-3ee23deaf5ae?source=---------601---------------

https://pesacheck.org/true-worlds-largest-plane-flew-for-the-first-time-on-april-13-2c346333fae6?source=---------961-----------------------
_actionscope_1926
https://pesacheck.org/mixture-muranga-senator-has-been-nominated-for-global-leadership-award-7cf067127fa2?source=---------962-----------------------
_actionscope_1928
https://pesacheck.org/false-there-is-no-evidence-that-the-man-pastor-lukau-resurrected-has-died-96ae70457888?source=---------963-----------------------
_actionscope_1930
https://pesacheck.org/false-there-was-no-train-collision-on-the-sgr-phase-2a-line-3df98ecd1eac?source=---------964-----------------------
_actionscope_1932
https://pesacheck.org/false-zambian-nurse-did-not-switch-5-000-babies-at-birth-fc4983b36171?source=---------965-----------------------
_actionscope_1934
https://pesacheck.org/true-unga-group-increased-maize-flour-prices-by-sh19-in-one-week-cc624ca56ba1?source=---------966-----------------------
_actionscope_1936
https://pesacheck.org/hoax-this-pho

In [84]:
no_duplicates = []

for link in pesa_links: 
    if link not in no_duplicates: 
        no_duplicates.append(link) 
        
len(no_duplicates)

1008

In [85]:
upper_frame = []

for link in no_duplicates:
    try:
        frame = []
        all_text = []

        url = link
        article_page = requests.get(url)
        soup = BeautifulSoup(article_page.text, "html.parser")

        title = soup.find('h1').text.strip()
        label = title.split(' ', 1)[0]
        title = title.split(' ', 1)[1]
        body = soup.find_all('p')
        body = body[:len(body)-8]

        for paragraph in body:
            text = paragraph.text.strip()
            all_text.append(text)
            
        text = collapse(all_text)
        print(title)
        frame.append([title,text,label])
        upper_frame.extend(frame)
        time.sleep(5)  
        
    except Exception as e:
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',url)
        print (error_type, 'Line:', error_info.tb_lineno)
    
    continue

This tweet of Kenyan businessman Buzeki criticising the Jubilee Party is fake
Joyful Women Organization is not taking loan applications through an online form
This photo of Kenya’s former Prime Minister, Raila Odinga, holding a Bible upside down, is doctored
This photo does not show men in Nigerian State Uniform
This advertisement for jobs at Café Javas Uganda is fake
This poster advertising jobs with the Kenya Defence Forces is not legitimate
tourism contribute 24% of Kenya’s GDP and employ 1.6 million people?
This Facebook page impersonating Kenya’s First Lady and offering cash grants is a scam
This poster with a quote alleged to have been published by Nation Africa has been fabricated
These photos do not show a truck razed by boda boda motorbike riders in Namanga, Kenya in January 2021
This post offering free mobile data from Safaricom is fake
This post offering Equity easy loans through an online form is fake
Ipsos Kenya has not released polls results on the popularity of senatoria

The traffic jam in this picture shows a scene from the US, not Kenya’s Thika highway
This image of the Saturday Standard November 14, 2020 edition is doctored
Margaret Kenyatta is not gifting Kenyans with KSh 20,000
The banner “Satan loves Kamala” on this image is doctored
This claim that Safaricom is giving KSh 7,000 cash gift to subscribers is a scam
This Google form loan application for ‘Waraka LoansKE’ is a scam
Deputy President Ruto has not travelled to the US to congratulate Trump
This poster calling for the arrest of Kenya’s Deputy President William Ruto is not from the police
This Google form advertising jobs at Carrefour is a scam
This photo of a muddy road is not from Nyandarua County, Kenya
This claim that Senator Gideon Moi is ranked last in a recent job performance poll is untrue
This image of Kenya’s Deputy President William Ruto is doctored
This Facebook page advertising jobs at Coca-Cola bottling company is not legitimate
This account showing a doctored photo of Joe Bid

This promotion for free gas cylinders from ProGas Kenya is fake
Former President Mwai Kibaki has not endorsed William Ruto’s candidacy for president in 2022
This site impersonating e-commerce platform Jumia is a scam
This post advertising thousands of jobs at USAID Kenya is fake
This company offering unsecured loans online is not legitimate
Jubilee Party has not endorsed Raila Odinga’s 2022 presidential bid
This Facebook post offering grants from Rachel Ruto is a hoax
The claim that Kenya’s deputy president assaulted a journalist is fabricated
This company offering loans through a Google form is not legitimate
This page giving away free laptops from HP is fake
This page using Family Bank’s Pesa Pap logo and offering loans through an online application form is fake
This page offering motorcycles from Uasin Gishu Women Representative Gladys Shollei is fake
This video does not show a robbery in an elevator at Sarit Centre, Nairobi
Kenya’s Teachers Service Commission is hiring 11,574 new t

This Facebook page offering loans to women and youth is not legitimate
Russia has not started production of a COVID-19 vaccine
FALSE: Kazakhstan is not facing an outbreak of pneumonia that is deadlier than COVID-19
This photo of travellers leaving Nairobi is not related to the lifting of COVID-19 lockdown orders
This infographic listing the health benefits of drinking water at specific times of the day is misleading
Something happened to Medium.
This Facebook page offering free laptops and phones from Cristiano Ronaldo is fake
cow urine be used as a disinfectant?
Infrared thermometers are not harmful to brain cells
2.6 million Kenyans have COVID-19 as reported by The Star?
This Facebook page offering Uwezo Fund loans is not legitimate
This ad for casual worker jobs at Coca-Cola is fake
WHO has not reversed its position on person-to-person transmission of COVID-19
This image does not show Kenyans rushing upcountry after the lockdown was lifted
A study has found that infectious droplets 

Kenya’s National Assembly has not scheduled a motion to impeach the deputy president
This statement from the Kenya Bureau of Standards on relief food distributed to Kikuyu residents is fake
Scientists in the US have not discovered a cure for COVID-19
Le ministre burundais de la santé n’est pas atteint du coronavirus
This Facebook post advertising jobs at Kenya Red Cross is fake
This Facebook page offering COVID-19 relief cash is fake
These photos of burnt trucks are not from Kiambu
This post claiming that Kenyan schools will reopen in August is fake
President Kenyatta has not appointed a post-COVID-19 economic recovery council
FALSE: Senegal has not authorized the use of COVID-Organics
President Andry Rajoelina did not receive money to add poison to COVID-Organics
19 truck drivers, not 182, tested negative for COVID-19 in Tanzania after testing positive in Kenya
This video of a riot is not linked to Kenya’s COVID-19 lockdown extension
There is no evidence that aspirin cures COVID-19
5G

This photo of citizens accosting a police officer is not from Nyamira
Kenya has recorded its first COVID-19 death
This newspaper front page cautioning against travel to Kenya due to COVID-19 is fake
These messages showing emergency cash transfers from the government of Kenya are fake
Governor Mike Sonko has not donated 58 million shillings to help fight COVID-19
Uber has not suspended night services in Kenya
Kenya has not confirmed 23 COVID-19 cases in Mombasa
Kilifi Deputy Governor Gideon Saburi has tested positive for COVID-19
Reports that a Chinese woman was killed in Mombasa over COVID-19 fears are false
This article claiming that a Kenyan has died from COVID-19 is false
This article claiming that a Kenyan has died from COVID-19 is a hoax
Kenya spending over 61 per cent of tax collections to repay debts?
Nairobi County has suspended arrests to prevent the spread of COVID-19
No motion has been tabled to impeach DP William Ruto
Public events have been banned in Kenya following confir

Governor Waiguru has demanded that King Kaka takes down ‘Wajinga Nyinyi’ song for defamation
Reports of 10 lions on the loose in Nairobi are exaggerated
A photo claiming to show musician King Kaka being arrested is from 2016
DP William Ruto has not been arrested by the EACC
This photo does not show the vehicle that plunged into the Indian Ocean on December 7
President Uhuru Kenyatta is the Kibabii University Chancellor
This image does not show UoN students protesting over Sonko’s arrest
This Letter Appointing Simon Mbugua as Deputy Governor is not from Sonko
Beatrice Elachi has not been sworn in as acting Nairobi County Governor
This photo of President Uhuru Kenyatta addressing a crowd is not from 2019
This ad for jobs at UNICEF Kenya is fake
This photo of a stray lion is from South Africa, not Kenya
The man in this photo is not KPC Managing Director Dr. Macharia Irungu
This post advertising jobs at Kenya Airways is fake
Meru County collect Sh540 million in the 2018/2019 financial year

Kenya just exported its first shipment of crude oil
A census enumerator was sexually assaulted in Kisumu
The World Bank did not withdraw from the James Gichuru-Rironi Road project over inflated costs
None of these photos shows destruction from the 2019 Amazon fires
Kenya gets the AU’s endorsement to seek UN Security Council Seat
Raila Odinga met and held talks with Indonesian President Joko Widodo
Sportpesa is yet to resume operations in Kenya
Othaya Level 6 has had its first ever cesarean section operation
Kenyan Chef Maliha Islam has surpassed the world record for longest time spent cooking nonstop
This ad for jobs at Total Kenya asking applicants to pay a fee is fake
much do we share online?
Former Prime Minister Raila Odinga has not announced that he will contest the presidency in 2022
Social media reports of Ezra Chiloba’s death are not true
This photo of an overturned petrol tanker is from 2012, not 2019
This image of a road in Maai Mahiu is actually from India
This site advertis

South African man did not dupe KFC into giving him free meals
Kenya has not been added to the U.S Visa Waiver Program
Canada has not ‘begged’ Kenya for one million immigrants
This photo of Kenyans queuing is actually from the April 2017 Jubilee Party Primaries
Nairobi County Government did not rehabilitate riparian land in Korogocho
Huduma Namba registration exercise has not been suspended
Kenyans who fail to acquire Huduma Namba will not be deported
Men in eSwatini have not been ordered to ‘marry more wives’ or face jail
Uganda benefited more than Kenya in bilateral trade since 2015?
Former Nairobi Governor has not been sentenced to hang
These images show the affordable housing project by the Kenyan government
Huduma Namba is not a requirement when applying for a Certificate of Good Conduct
The claim that university students should wear uniform is a hoax
Kenyatta University Student Portal database has not been hacked into
This image claiming to show Gideon Moi at DP Ruto’s Karen home 

In [87]:
len(upper_frame)

1007

In [88]:
data = pd.DataFrame(upper_frame, columns=['Title','Text','Label'])
data.head()

,Title,Text,Label
0,This tweet of Kenyan businessman Buzeki critic...,A screenshot shared on Facebook claiming to sh...,FALSE:
1,Joyful Women Organization is not taking loan a...,A claim made on Facebook that the Joyful Women...,HOAX:
2,"This photo of Kenya’s former Prime Minister, R...",A photo shared on Facebook showing former Prim...,ALTERED:
3,This photo does not show men in Nigerian State...,This tweet with an image published on January ...,FALSE:
4,This advertisement for jobs at Café Javas Ugan...,A Facebook post claiming that Café Javas Ugand...,HOAX:


In [89]:
path = '/home/jaden/Development/Notebooks/kenyan-news'
data.to_csv(os.path.join(path,r'pesa-check.csv'))

## Star News Real News

In [100]:
pages = list(range(1, 76))
star_links = []
number_of_links = 0

for page in pages:
    print(number_of_links)
    url = 'https://www.the-star.co.ke/news/?limit=10&page=' + str(page) +'&partial=true'
    page = requests.get(url).json()
    soup = BeautifulSoup(page["html"], "html.parser")
    articles = soup.find_all('div',attrs={'class':'section-article section-article-has-comments'})

    for article in articles:
        link = article.find('div', attrs = {'class': 'article-body'}).find('a', href=True)
        star_links.append(link['href'])
        number_of_links += 1
        
    time.sleep(7)

0
13
20
32
44
55
65
76
88
100
110
119
132
142
152
164
175
185
196
207
216
230
243
255
266
275
288
297
304
312
322
339
353
364
375
384
393
406
416
429
439
452
461
475
485
497
511
518
529
541
552
565
577
591
605
618
630
641
653
666
672
684
693
703
715
729
740
754
768
783
797
810
827
842


KeyboardInterrupt: 

In [101]:
len(star_links)

no_duplicates = []

for link in star_links: 
    if link not in no_duplicates: 
        no_duplicates.append(link) 
        
len(no_duplicates)

842

In [102]:
no_duplicates

['/news/2021-02-01-betty-kyalo-boniface-mwangi-kipchoge-feted-in-digital-influencer-awards/',
 '/news/2021-02-01-brown-bottle-why-overcoming-alcoholism-is-a-struggle/',
 '/news/2021-02-01-ugandan-elections-mighty-salim-and-larry-king-top-google-searches/',
 '/news/2021-02-01-uhuru-leads-kenyans-in-mourning-former-minister-nyachaes-death/',
 '/news/2021-02-01-unrest-in-schools-likely-fuelled-by-drug-abuse-nacada/',
 '/news/2021-02-01-update-mwilu-gets-reprieve-as-court-halts-order-barring-her-from-office/',
 '/news/2021-02-01-britams-wairegi-hands-over-leadership-to-tavaziva-madzinga/',
 '/news/2021-02-01-dci-probes-suspected-murder-of-bungoma-county-worker/',
 '/news/2021-02-01-fighter-powerful-this-man-simeon-nyachae/',
 '/news/2021-02-01-mt-kenya-jumia-customers-to-benefit-from-reduced-delivery-fees/',
 '/news/2021-02-01-stop-insulting-uhuru-lets-have-decorum-kabogo-tells-politicians/',
 '/news/2021-02-01-icc-officially-releases-lawyer-paul-gicheru-from-custody/',
 '/news/2021-02-01-

In [107]:
upper_frame = []
counter = 0 

for link in no_duplicates:
    try:
        frame = []
        all_text = []
        link_url = 'https://www.the-star.co.ke' + link
        article_page = requests.get(link_url)
        soup = BeautifulSoup(article_page.text, "html.parser")
        title = soup.find('h1',attrs={'class':'article-title article-title-primary'}).text.strip()
        texts = soup.find_all('div',attrs={'class':'article-widget article-widget-text'})
        for x in texts:
            text_body = x.text.strip()
            all_text.append(text_body)

        text = collapse(all_text)
        print(counter, title)
        frame.append([title,text])
        upper_frame.extend(frame)
        counter += 1
        time.sleep(7)    
        
    except Exception as e:
        time.sleep(15)
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',link_url)
        print (error_type, 'Line:', error_info.tb_lineno)
    
    continue

0 Betty Kyalo, Boniface Mwangi, Kipchoge feted in digital influencer awards
1 Brown bottle: Why overcoming alcoholism is a struggle
2 Ugandan elections, Mighty Salim and Larry King top Google searches
3 Uhuru leads Kenyans in mourning former minister Nyachae's death
4 Unrest in schools likely fuelled by drug abuse — Nacada
5 [UPDATE] Mwilu gets reprieve as court halts order barring her from office
6 Britam's Wairegi hands over leadership to Tavaziva Madzinga
7 DCI probes suspected murder of Bungoma county worker
8 'Fighter, powerful': This man Simeon Nyachae
9 Mt Kenya Jumia customers to benefit from reduced delivery fees
10 Stop insulting Uhuru, let’s have decorum, Kabogo tells politicians
11 ICC officially releases lawyer Paul Gicheru from custody
12 Uhuru postpones Sagana meeting with Mt Kenya clergy, women leaders
13 MPs differ on best how to deal with Kapedo bandits
14 Cops? Caning? How about God to end school unrest?
15 Women at high risk of HIV can use vaginal ring - WHO
16 Spea

135 Uhuru, Raila in epic battle against Ruto as IEBC clears BBI bill
136 Kikuyu, Kalenjins grossly overrepresented in civil service
137 From convicting accused persons as magistrate to being a lawyer
138 Ruto's sloganeering will take Kenya to the dogs, says Raila
139 More than 286,000 officers to man KCPE, KCSE exams
140 Six more die of Covid, 27 on oxygen
141 Bryan Khaemba: Why I don't miss the bench
142 Kenyan journalists to benefit from WAN-IFRA's media freedom programme
143 Kibwana quits CoG chair race to focus on presidency
144 I still have time to roar like a lion, Uhuru tells critics
145 Police are not your enemies - Uhuru tells Kenyans
146 Court bars Shoprite supermarkets from leaving Kenya
147 Military leaves Wine’s home, but choppers still hover around
148 Uhuru commissions construction of police academy, 210 housing units in Ngong
149 Six deaths as 141 new Covid cases recorded
150 IEBC verifies 1.14 million BBI signatures
151 Nyati barracks opened as Kenya, UK strengthen tie

263 IEBC official among the six found dead in Tiaty, Baringo
264 JKIA-Westlands expressway has employed 2,000 people - PDU
265 BBI signatures mystery: Uhuru, Raila names missing
266 Former KPA MD Manduku enters Kisii Governor race
267 Relief as Treasury releases Sh24 billion to counties
268 Kenya reports two cases of new virus variant seen in South Africa
269 KDF warns of circulation of fake calling letters ahead of recruitment drive
270 IEBC publishes verified BBI signatures for public scrutiny
271 Biden sets to work on reversing Trump policies with executive orders
272 Embakasi woman who splashed man with acid charged
273 Are you under surveillance? Why people are fleeing WhatsApp
274 MPs want action against leaders behind Northeastern violence
275 Magoha lauds schools for keeping virus at bay
276 County workers' union seeks to block construction of Sh5bn tower
277 Religious leaders volunteer to arbitrate in health sector dispute
278 Trader who shot Ronalo eatery boss apologises for 

394 UDA issues certificate for Machakos senator by-election
395 A military man among two charged with causing grievous harm
396 DPP to lose security of tenure in BBI plan, Maraga warns
397 Sh34bn 'lying idle' in CBK vault tied to projects — governors
398 NLC plans to set aside land for public projects
399 Sonko's letter withdrawing my nomination was blank: Kananu
400 Cop who shot friend released
401 [PHOTOS] Ruto meets ex-governor Rutto at Bomet rally
402 Anne Kananu vetting: Court declines to issue stay orders
403 Museveni takes early lead in Uganda elections
404 Former MP aspirant vows to turn tide against Kalonzo for Ruto
405 Ritual killing? Embu pupil says he slaughtered cousin, drank blood
406 Ruto’s 2022 bid in limbo as Uhuru jolts DP’s game plan
407 Registrar blocks registration of at least 176 new parties
408 IEBC denies sabotaging BBI signature verification
409 Ex-governor Rutto to join DP's camp at Bomet event
410 Ministry projects 42 million bags of maize harvest
411 Spend S

528 Retract statement over tribal leadership - Murkomen urges Uhuru
529 Why Igathe’s way of resignation is a non-issue to IEBC
530 Open letter to Nairobi governor by-election candidates
531 Kenyans must reject populist conmanship
532 Justice Njoki Ndungu car still missing after being stolen in Nairobi
533 Indonesia Boeing 737 passenger plane crash site found, navy says
534 Murder weapon recovered as son confesses to killing family members in Kiambu incident
535 Sonko to take a 10-year break from politics
536 Baba Yao: An opportunist, a strategist or both?
537 Children not yet back to school should report - Matiang'i
538 Student fakes his kidnapping, demands ransom
539 Indonesian passenger plane missing after take-off
540 Tea Board of Kenya will be up and running by April - CS Munya
541 Vandals steal Sh500,000 road signs, lights, railings
ERROR FOR LINK: https://www.the-star.co.ke/counties/western/2021-01-09-well-hunt-down-pregnant-girls-cs-magoha/
<class 'requests.exceptions.Connection

656 Kenya takes its position in the UN Security Council for the 2021-2022 term
657 Machakos senatorial by-election date revised to March 18, from March 23
658 Yemeni boy, ravaged by hunger, weighs 7 kg
659 Sonko wins round one as court suspends Nairobi by-election
660 Kenya likely to wait until March for Covid-19 vaccine as India bans export
661 BBI will resolve historical injustices, says Raila
662 Kang'ata to Uhuru: BBI is unpopular in Mt Kenya
663 ODM bigwigs tell Ruto and allies to respect Uhuru, Raila
664 No fatality as Kenya maintains positivity rate below 5%
665 Fimbo ya Nyayo: Will Gideon Moi restore Kanu to its former glory?
666 Three days till nurses strike referral hospitals
667 PSV drivers get AlcoBlow tests as schools open
668 Three Nigerians arrested over the disappearance of a teen girl
669 Locust spraying fails in Kitui, residents yell, beat pans
670 Government rehabilitates idle, dilapidated railway line
671 ODM’s era in the coastal region has been catastrophic: Ruto a

788 Tantalising foods to treat your family to this festive season
789 Covid-19: The uninvited guest that brought the world to a standstill
790 Impeachment: A fair process or a political weapon?
791 [PHOTOS] Uhuru lights up Nairobi's iconic fig tree for Christmas
792 Uhuru asks Africa CDC to issue harmonised Covid protocols for AU meetings
793 Ngilu condemns act where man allegedly burnt family to death
794 Nurses still on strike! Panyako says as he hits out at doctors
795 Leaders pledge support for Nyaribo as Nyagarama laid to rest
796 Kenya records 236 new cases as four succumb to disease
797 Busia County has vacancies for health workers - Ojaamong
798 KIWASH injects Sh63m to Busia for efforts in combating Covid-19
799 Bunyenyezi, man who paid Raila's air ticket to exile in Norway dies aged 76
800 Stop threatening to sack medics, MP Alice Wahome tells Kagwe
801 NCIC brokers truce between Mandera's warring clans
802 Final send-off: Raila, Matiang'i lead mourners in paying last respects

In [108]:
data = pd.DataFrame(upper_frame, columns=['Title','Text'])
data.head()

,Title,Text
0,"Betty Kyalo, Boniface Mwangi, Kipchoge feted i...","Sports personality Eliud Kipchoge, TV personal..."
1,Brown bottle: Why overcoming alcoholism is a s...,"It's brown, it's colourless, it comes in diffe..."
2,"Ugandan elections, Mighty Salim and Larry King...",Ugandan Elections and the deaths of renowned ‘...
3,Uhuru leads Kenyans in mourning former ministe...,President Uhuru Kenyatta has joined leaders in...
4,Unrest in schools likely fuelled by drug abuse...,The cases of unrest in schools might be a resu...


In [109]:
path = '/home/jaden/Development/Notebooks/kenyan-news'
data.to_csv(os.path.join(path,r'real-news.csv'))